In [81]:
%load_ext jupyter_black

In [8]:
import os
import site
from pathlib import Path
"/opt/venv/lib/python3.10/site-packages/keras/__init__.py"
# site.getsitepackages()
# tf_path = Path("/opt/venv/lib64/python3.10/site-packages/tensorflow/__init__.py")
# with tf_path.open("rt") as f:
#     lines = f.readlines()
# [line for line in lines if line.startswith(("from", "import"))]
# k_path = Path("/opt/venv/lib64/python3.10/site-packages/keras/api/_v2/__init__.py")
# with k_path.open("rt") as f:
#     lines = f.readlines()
# [line for line in lines if line.startswith(("from", "import"))]
# keras_walk = os.walk("/opt/venv/lib64/python3.10/site-packages/keras/api/_v2/")
# STUBDIR = Path("/home/leaver2000/griblib/stubs/tensorflow/keras")


def private(filename: str) -> bool:
    return filename.startswith("__") or filename.endswith("__")


def __write_all(lines: list[str]):
    for line in lines:
        if line.startswith(("from", "import")):
            line = line.strip().split()
            if not line[-1].startswith("_"):
                yield f'"{line[-1]}"'


def __find_docstring(lines: list[str]) -> slice:
    idx1 = None
    for i, line in enumerate(lines):
        if line.startswith('"""'):
            idx1 = i

        if idx1 and line.endswith('"""\n'):
            return i + 2
    return 5


def write_keras(stub_out_dir: str, keras_api_dir: str) -> None:
    stubpath = Path(stub_out_dir)
    __full_all = ""
    if not stubpath.exists():
        stubpath.mkdir(parents=True)
    pub_modules =[]
    for directory, modules, filenames in os.walk(keras_api_dir):
        for module in modules:
            if not private(module) and not private(directory):
                pub_modules.append(module)
                src = f"{directory}/{module}/{filenames[0]}"
                newstub = stubpath / f"{module}/{filenames[0]}i"
                newdir = stubpath / module
                if not newdir.exists():
                    newdir.mkdir(parents=True)
                if not newstub.exists():
                    newstub.touch()

                with open(src, "rt") as fin:
                    with newstub.open("wt") as fout:
                        lines = fin.readlines()
                        _all = ",\n".join(__write_all(lines))
                        __full_all += _all
                        _all = f"__all__ = [\n{_all}\n]\n\n"
                        idx = __find_docstring(lines)
                        # if idx:
                        lines.insert(idx, _all)

                        fout.write("\n".join(lines))
    _initstub= Path(stub_out_dir  ).parent / "keras.pyi"

    if not _initstub.exists():
        _initstub.touch()
    with _initstub.open("wt") as final:
        all_modules =',\n'.join(pub_modules)
        final.write(f"from keras import ({all_modules})")
        # final.write(f"__all__ = [\n{__full_all}\n]\n\n from keras import ({all_modules})")


write_keras(
    "/home/leaver2000/griblib/stubs/tensorflow/keras",
    "/opt/venv/lib64/python3.10/site-packages/keras/api/_v2/",
)

In [9]:
import tensorflow.keras as tfk # type: ignore
from tensorflow import keras
tfk.Sequential


keras.engine.sequential.Sequential